In [11]:
import PyPDF2
import glob
import numpy as np
import pdfplumber
import pandas as pd
from collections import defaultdict
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure

In [12]:
def text_extraction(element):
    
    line_text = element.get_text()
    
    line_formats = []
    for text_line in element:
        
        if isinstance(text_line, LTTextContainer):
            
            for character in text_line:
                
                if isinstance(character, LTChar):
                    
                    line_formats.append(character.fontname)
                    
                    line_formats.append(character.size)
                    
    format_per_line = list(set(line_formats))
    
    return (line_text, format_per_line)

def analyst_executives(path):
    
    time=0
    presentation_index=0
    Elon_Musk_index=0
    Question_and_Answer_index=0
    Elon_Musk_speech=[]
    presentation_speech=[]
    text=[]
    executives_name=[]
    analysts_name=[]

    executives_title=[]
    analysts_title=[]

    executives_index=0
    analysts_index=0

    for pagenum, page in enumerate(extract_pages(path)):
        
        for _, element in enumerate(page):

            if pagenum == 2:

                if isinstance(element, LTTextContainer):
                    line_text, format_per_line = text_extraction(element)

                text.append(line_text)

                if line_text == "EXECUTIVES\n":
                    executives_index+=1

                if line_text == 'ANALYSTS\n':
                    executives_index=0

                if executives_index >= 1:
                    executives_name.append(line_text.split("\n")[0])
                    executives_title.append(line_text.split("\n")[1])

                if line_text == 'ANALYSTS\n':
                    analysts_index+=1
                    executives_index=0

                if 'S&P Global Market Intelligence' in line_text:
                    analysts_index=0

                if analysts_index >= 1:
                    analysts_name.append(line_text.split("\n")[0])    
                    analysts_title.append(line_text.split("\n")[1])

        if pagenum == 4:
            break

    ppl=defaultdict(lambda: defaultdict(str))

    for i in range(1, len(analysts_name), 1):

        ppl['analysts'][analysts_name[i]]=analysts_title[i]

    for i in range(1, len(executives_name), 1):

        ppl['executives'][executives_name[i]]=executives_title[i]
    
    return ppl

In [36]:
def processing_qa(path):
    
    presentation_index=0
    Elon_Musk_index=0
    Question_and_Answer_index=0
    Elon_Musk_speech=[]
    question_and_answer_speech=[]
    time=0
    
    pdf=open(path, 'rb')
    pdfReader=PyPDF2.PdfReader(pdf)

    for pagenum, page in enumerate(extract_pages(path)):
        
        print('page={}'.format(pagenum, page))
        
        for _, element in enumerate(page):
            
            if isinstance(element, LTTextContainer):

                line_text, format_per_line = text_extraction(element)
            
            if pagenum == 0:
                
                if _ == 0:
                    
                    time=line_text
                    time=time[61:100]
            
            if "TESLA, INC."  in line_text:
        
                pass
        
            elif "S&P Global Market Intelligence" in line_text:
        
                pass
            
            elif len(line_text) < 6:
        
                pass
        
            else:

                if line_text == 'Presentation\n':

                    presentation_index = presentation_index + 1

                if line_text == 'Question and Answer\n':

                    Question_and_Answer_index = Question_and_Answer_index + 1

                if Question_and_Answer_index == 2:

                    question_and_answer_speech.append(line_text)

                if pagenum == pdfReader.pages.length_function()-1:
                               
                    break
                
    return time, question_and_answer_speech

In [37]:
def questions_answers_dict(question_and_answer_speech, executives_people, analysts_people):

    speech_analysts_people=defaultdict(lambda: defaultdict(str))
    speech_executives_people=defaultdict(lambda: defaultdict(str))
    speech_analysts_index_dict=defaultdict(lambda: defaultdict(str))
    speech_executives_index_dict=defaultdict(lambda: defaultdict(str))
    speech_question_index=0
    speech_answer_index=0
        
    for _, text in enumerate(question_and_answer_speech):

        text=text.split("\n")

        if "S&P Global Market Intelligence" in text[0]:

            pass

        else:

            for analysts in analysts_people:
                                
                if analysts in text[0]:
                                        
                    speech_analysts_people[str("question_")+str(speech_question_index)]=analysts
                    speech_analysts_index_dict[str("question_")+str(speech_question_index)]=_+1
                    speech_question_index+=1
                        
            for executives in executives_people:

                if executives in text[0]:

                    speech_executives_people[str("answer_")+str(speech_answer_index)]=executives
                    speech_executives_index_dict[str("answer_")+str(speech_answer_index)]=_+1
                    speech_answer_index+=1
    
    # ================================================================================ 
    
    speech_speech_executives_index_list=[]

    for i in speech_executives_index_dict.items():

        speech_speech_executives_index_list.append(i[1])

    speech_speech_executives_index_list=np.array(speech_speech_executives_index_list)+1

    # ================================================================================
    
    speech_speech_analysts_index_list=[]

    for i in speech_analysts_index_dict.items():
            
        speech_speech_analysts_index_list.append(i[1])

    speech_speech_analysts_index_list=np.array(speech_speech_analysts_index_list)+1
        
    # ================================================================================
    
    speech_speech_executives_dict=defaultdict(lambda: defaultdict(list))

    for i in speech_executives_index_dict.items():
        
        if i[1] == len(question_and_answer_speech):
            break
            
        speech_speech_executives_dict[i[0]][i[1]].append(question_and_answer_speech[i[1]])
    
    # ================================================================================ 
    
    speech_speech_analysts_dict=defaultdict(lambda: defaultdict(list))

    for i in speech_analysts_index_dict.items():
        
        if i[1] == len(question_and_answer_speech):
            break
            
        speech_speech_analysts_dict[i[0]][i[1]].append(question_and_answer_speech[i[1]])
    
    
    return speech_speech_executives_dict,\
            speech_speech_analysts_dict,\
            speech_executives_people,\
            speech_analysts_people


In [38]:
def data_processing_qa(path):
    
#     pdf=open(path, 'rb')
#     pdfReader=PyPDF2.PdfReader(pdf)

    time, question_and_answer_speech=processing_qa(path)

    ppl=analyst_executives(path)

    ppl_reverse={v:k  for k,v in ppl['executives'].items()}

    for i in ppl_reverse:

        if "Investor" in i:

            ppl['analysts'][ppl_reverse[i]]=i

            ppl['executives'].pop(ppl_reverse[i], None)

    executives_people=[elem[0] for elem in ppl['executives'].items()]
    executives_title=[elem[1] for elem in ppl['executives'].items()]

    analysts_people=[elem[0] for elem in ppl['analysts'].items()]
    analysts_title=[elem[1] for elem in ppl['analysts'].items()]
    
    speech_speech_executives_dict,\
    speech_speech_analysts_dict,\
    speech_executives_people,\
    speech_analysts_people=\
    questions_answers_dict(question_and_answer_speech,
                           executives_people,
                           analysts_people)
    
    return time, path.split("/")[-1], ppl,\
            question_and_answer_speech,\
            speech_speech_executives_dict,\
            speech_speech_analysts_dict,\
            speech_executives_people,\
            speech_analysts_people

In [39]:
files=glob.glob("/Volumes/4TB/ConferenceCall/Tesla/Earningcall/*.pdf")
path=files[-3]
print(path)

/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla, Inc., Q4 2021 Earnings Call, Jan 26, 2022.pdf


In [42]:
data=defaultdict(lambda: defaultdict(dict))

for path in files[:2]:
    print(path)
    time, document, ppl,\
    question_and_answer_speech,\
    speech_speech_executives_dict,\
    speech_speech_analysts_dict,\
    speech_executives_people,\
    speech_analysts_people=data_processing_qa(path)
    
    data[time][document]["people"]=ppl
    data[time][document]["question_and_answer"]=question_and_answer_speech
    data[time][document]["executives_answer"]=speech_speech_executives_dict
    data[time][document]["analysts_question"]=speech_speech_analysts_dict
    data[time][document]["executives_people_answer"]=speech_executives_people
    data[time][document]["analysts_people_question"]=speech_analysts_people

/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2011 Earnings Call, May 04, 2011.pdf
page=0
page=1
page=2
page=3
page=4
page=5
page=6
page=7
page=8
page=9
page=10
page=11
page=12
page=13
page=14
page=15
page=16
page=17
page=18
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2012 Earnings Call, May 09, 2012.pdf
page=0
page=1
page=2
page=3
page=4
page=5
page=6
page=7
page=8
page=9
page=10
page=11
page=12
page=13
page=14
page=15
page=16
page=17
page=18
page=19


In [31]:
data_table=\
pd.DataFrame([
              col1, col2, col3, d3
             ]
              for col1, d1 in data.items()\
              for col2, d2 in d1.items()\
              for col3, d3 in d2.items()
            )

In [32]:
data_table=data_table.rename(columns={0: 'time', 1: 'company', 2: 'chara', 3: 'text'})

In [33]:
data_table#

,time,company,chara,text
0,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",people,{'analysts': {'Adam Michael Jonas': 'Morgan St...
1,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",question_and_answer,"[Question and Answer\n, Operator\n, [Operator ..."
2,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",executives_answer,"{'answer_0': {6: ['Sure, our goal with Tesla t..."
3,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",analysts_question,{'question_0': {4: ['So I guess a couple quest...
4,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",executives_people_answer,"{'answer_0': 'Elon R. Musk', 'answer_1': 'Elon..."
5,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",analysts_people_question,"{'question_0': 'Patrick Kenehan Archambault', ..."


In [68]:
table=\
pd.pivot(data_table,
         values=['text'],
         index=['time', 'company'],
         columns=['chara'])

In [81]:
table_=table['text'].reset_index(drop=False)
table_["date"]=table_["time"].apply(lambda x: x.split(',')[1])
table_["year"]=\
table_["time"].apply(lambda x: x.split(',')[2][0:5])
table_["datetime"]=\
table_["time"].apply(lambda x: x.split(',')[2][5:11])
table_["quarter"]=\
table_["company"].apply(lambda x: x.split(',')[2][:3])
table_["year_quarter"]=\
table_["company"].apply(lambda x: x.split(',')[2][3:9])
table_=table_.sort_values(['year_quarter', 'quarter'])

In [177]:
table_.columns

Index(['date', 'datetime', 'year', 'year_quarter', 'quarter', 'time',
       'company', 'analysts_people_question', 'analysts_question',
       'executives_answer', 'executives_people_answer', 'people',
       'question_and_answer'],
      dtype='object', name='chara')

In [162]:
table_=\
table_[['date', 'datetime', 'year', 'year_quarter', 'quarter',
        'time', 'company', 'analysts_people_question', 'analysts_question',
        'executives_answer', 'executives_people_answer', 'people',
        'question_and_answer']].reset_index(drop=True)

In [175]:
path_output=\
'/Volumes/4TB/ConferenceCall/Tesla/csvfile/tesla_earning_call_question_answer.csv'

In [176]:
table_.to_csv(path_output, index = False)